In [2]:
# Imports
import os
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

In [63]:
data = pd.read_csv('../data/3m.tsv', delimiter='\t')

/opt/conda/envs/reaction-workbench-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3135: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,16,17,18,19,20,21,22,23,24,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [64]:
df=data

### Remove Big Reactions

In [68]:
# Filter out rows where the number of reactants or products is more than 2
df = df[(df['#reactants'] <= 2) & (df['#products'] <= 2)]

In [66]:
df_bigreactions = df[(df['#reactants'] >= 3) | (df['#products'] >= 3)]

In [8]:
#df_bigreactions.to_csv('big_reactions.csv', index=False)

### Remove Rare Reactions

In [69]:
# Convert multi Class-ID into one
def clean_class_id(row):
    # Split the string by comma and convert to a list
    class_ids = str(row['CLASS-ID']).split(',')
    # Return the first element from the list, ensuring it's an integer
    return int(class_ids[0].strip())

# Apply the function to the 'CLASS-ID' column
df['CLASS-ID'] = df.apply(clean_class_id, axis=1)

/tmp/ipykernel_7687/1979647533.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CLASS-ID'] = df.apply(clean_class_id, axis=1)


In [70]:
df_exploded = df.explode('CLASS-ID')

In [71]:
#Count the occurrences of each CLASS-ID
class_counts = df_exploded['CLASS-ID'].value_counts()
rare_classes = class_counts[class_counts < 10].index.tolist()

In [72]:
# Filter out rows with rare CLASS-IDs
df_filtered = df_exploded[~df_exploded['CLASS-ID'].isin(rare_classes)]

In [73]:
df_rare= df_exploded[df_exploded['CLASS-ID'].isin(rare_classes)]

In [15]:
# Convert CLASS-ID back to a string if it's currently a list
#df_filtered['CLASS-ID'] = df_filtered['CLASS-ID'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

### Combine reactants with products

In [7]:
def combine_reactants_products(row):
    reactants = '.'.join(filter(lambda x: not pd.isnull(x), [row['REACTANT1'], row['REACTANT2']]))
    products = '.'.join(filter(lambda x: not pd.isnull(x), [row['PRODUCT1'], row['PRODUCT2']]))
    return f"{reactants}>>{products}"

In [19]:
df_filtered['reaction'] = df_filtered.apply(combine_reactants_products, axis=1)

/tmp/ipykernel_7687/134776504.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['reaction'] = df_filtered.apply(combine_reactants_products, axis=1)


### Remove unrelevant coloumns

In [21]:
df_total=df_filtered[['CLASS-ID','RX.ID','REACTANT1','REACTANT2','PRODUCT1','PRODUCT2','reaction']]

In [22]:
df_total['product_summary'] = df_total['PRODUCT1'].fillna('') + df_total['PRODUCT2'].fillna('')

/tmp/ipykernel_7687/3592978022.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total['product_summary'] = df_total['PRODUCT1'].fillna('') + df_total['PRODUCT2'].fillna('')


In [23]:
df_presplit=df_total[['CLASS-ID','RX.ID','reaction','product_summary']]

### Show some info about the reactions with same products

In [24]:
unique_products = df_presplit.groupby('product_summary').ngroup()

In [26]:
product_counts = df_presplit.groupby('product_summary').size()

In [57]:
df_presplit.duplicated(subset=['CLASS-ID', 'product_summary']).sum()

193666

In [58]:
df_presplit.duplicated(subset=['product_summary']).sum()

298087

### Split dataset

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
unique_products = df_presplit['product_summary'].unique()

In [48]:
stratify_df = df_presplit.drop_duplicates(subset=['CLASS-ID', 'product_summary'])

In [53]:
# Stratify by 'CLASS-ID' to ensure at least one example of every class in the training/test/val set.
# Split the stratify_df into training+val and test datasets for further splitting.
trainAndVal_df, test_df = train_test_split(
    stratify_df, 
    test_size=0.1,
    stratify=stratify_df['CLASS-ID'],
    random_state=27
)

In [54]:
# Split the temporary set into validation and test sets.
train_df, val_df = train_test_split(
    trainAndVal_df,
    test_size=1/9, 
    stratify=trainAndVal_df['CLASS-ID'],
    random_state=27
)

In [59]:
len(train_df), len(val_df), len(test_df)

(2209000, 276125, 276125)

In [60]:
# Assign the rest of the data with the same products to the corresponding sets.
train_df = df_presplit[df_presplit['product_summary'].isin(train_df['product_summary'])]
val_df = df_presplit[df_presplit['product_summary'].isin(val_df['product_summary'])]
test_df = df_presplit[df_presplit['product_summary'].isin(test_df['product_summary'])]

# The lengths of each dataset will give us the distribution.
len(train_df), len(val_df), len(test_df)

(2405606, 331181, 330810)

In [39]:
#Make sure all the sets have all the classes
print(train_df['CLASS-ID'].nunique())
print(val_df['CLASS-ID'].nunique())
print(test_df['CLASS-ID'].nunique())

1299
1299
1299


In [40]:
train_df['CLASS-ID'].value_counts()

1085    185024
432     164914
1016    113635
60       85921
433      60554
         ...  
1345         8
1138         8
1404         7
500          7
1622         7
Name: CLASS-ID, Length: 1299, dtype: int64

In [41]:
test_df['CLASS-ID'].value_counts()

1085    24966
432     21259
1016    15507
60      11185
433      7977
        ...  
881         1
500         1
622         1
1133        1
121         1
Name: CLASS-ID, Length: 1299, dtype: int64

In [43]:
train_df.to_csv('../data/train.csv', index=False)
val_df.to_csv('../data/val.csv', index=False)
test_df.to_csv('../data/test.csv', index=False)

In [5]:
rare_reactions = pd.read_csv('../data/rare_reactions.csv', delimiter=',')

In [8]:
rare_reactions['reaction'] = rare_reactions.apply(combine_reactants_products, axis=1)

In [9]:
rare_reactions=rare_reactions[['CLASS-ID','RX.ID','REACTANT1','REACTANT2','PRODUCT1','PRODUCT2','reaction']]
rare_reactions=rare_reactions[['CLASS-ID','RX.ID','reaction']]

In [11]:
rare_reactions.to_csv('../data/rare.csv', index=False)